In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\chest-cancer-classification'

In [3]:
import tensorflow as tf
MODEL = tf.keras.models.load_model("artifacts/model_training/model.keras")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\chest-cancer-classification\venv\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\chest-cancer-classification\venv\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\chest-cancer-classification\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\chest-cancer-classification\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
 

AttributeError: _ARRAY_API not found

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    trainig_data: Path
    all_params:  dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [5]:
from chest_cancer.constants import *
from chest_cancer.utils.commen import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        model_evaluation_config = ModelEvaluationConfig(
          model_path = "artifacts/model_training/model.keras",
          trainig_data = "artifacts/data_ingestion/chest-cancer-ct",
          mlflow_url = os.environ["MLFLOW_EXPERIMENT_URL"],
          all_params = self.params,
          params_image_size = self.params.IMAGE_SIZE,
          params_batch_size = self.params.BATCH_SIZE
        )

        return model_evaluation_config

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
dagshub.init(repo_owner='ajuarjun528', repo_name='chest-cancer-classification', mlflow=True)

2025-01-22 19:20:23,505 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as ajuarjun528

2025-01-22 19:20:23,514 - INFO - helpers - Accessing as ajuarjun528
2025-01-22 19:20:23,945 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/repos/ajuarjun528/chest-cancer-classification "HTTP/1.1 200 OK"
2025-01-22 19:20:24,407 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ajuarjun528/chest-cancer-classification"

2025-01-22 19:20:24,412 - INFO - helpers - Initialized MLflow to track repo "ajuarjun528/chest-cancer-classification"


Repository ajuarjun528/chest-cancer-classification initialized!

2025-01-22 19:20:24,415 - INFO - helpers - Repository ajuarjun528/chest-cancer-classification initialized!


In [8]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.trainig_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = MODEL.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

2025-01-22 19:20:24,478 - INFO - commen - yaml file: config\config.yaml loaded successfully
2025-01-22 19:20:24,483 - INFO - commen - yaml file: params.yaml loaded successfully
2025-01-22 19:20:24,485 - INFO - commen - Created directory at artifacts
Found 102 images belonging to 2 classes.


d:\chest-cancer-classification\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8068 - loss: 1.1126
2025-01-22 19:20:35,524 - INFO - commen - Saved json file at scores.json


2025/01/22 19:20:37 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 19:20:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/22 19:21:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


🏃 View run brawny-hawk-534 at: https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow/#/experiments/0/runs/5f24006ca0dc495785e20cf7e631a501
🧪 View experiment at: https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow/#/experiments/0
